In [1]:
import glob
import nibabel
import numpy as np
import re
import pandas as pd
from collections import defaultdict
from matplotlib import pyplot as plt
from tqdm import tqdm
from IPython.core.debugger import set_trace
import os
import cc3d
import shutil
from collections import defaultdict
import pickle

import torch
from torch import nn
from torch import autograd
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from utils import show_slices, check_patch, get_symmetric_value

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important}</style>"))

`fcd_data/normalized_segmentation` -   
`fcd_data/normalized_data` -   
`fcd_data/normalized_label` - 
`fcd_data/fmriprep` - from Ironman server, 

In [2]:
paths_dict = defaultdict(dict)
for p in os.listdir('../fcd_data/normalized_label'):
    
    label = p.split('.')[0]
    
    sub_root = f'../fcd_data/normalized_data/sub-{label}/anat/'
    brain_path = glob.glob(os.path.join(sub_root, '*Asym_desc-preproc_T1w.nii.gz'))[0]
    mask_path = glob.glob(os.path.join(sub_root, '*Asym_desc-brain_mask.nii.gz'))[0]
    label_path = f'../fcd_data/normalized_label/{p}'
    
    paths_dict[label]['label'] = label_path
    paths_dict[label]['brain'] = brain_path    
    paths_dict[label]['mask'] = mask_path  
    
#     # features
#     thickness_path = f'../fcd_data/preprocessed_data_anadezhda/thickness/norm-{label}.nii'
#     sulc_path = f'../fcd_data/preprocessed_data_anadezhda/sulc/norm-{label}.nii'
#     curv_path = f'../fcd_data/preprocessed_data_anadezhda/curv/norm-{label}.nii'
    

    
#     paths_dict[label]['thickness'] = thickness_path
#     paths_dict[label]['sulc'] = sulc_path    
#     paths_dict[label]['curv'] = curv_path    

In [3]:
# label_tensors = []

centers = {}
cc3d_dict = {}
d_s = {}

label_info = defaultdict(dict)

for label, pdict in tqdm(paths_dict.items()):
    
    mask_tensor = nibabel.load(pdict['mask']).get_fdata() > 0
    mask_tensor = mask_tensor.astype(int)
#     brain_tensor = nibabel.load(pdict['brain']).get_fdata()
#     brain_tensor = brain_tensor*mask_tensor.astype(int)

    label_tensor = nibabel.load(pdict['label']).get_fdata() * mask_tensor
    label_tensor = np.array(label_tensor) > 0
    
    labels_out = cc3d.connected_components(label_tensor, connectivity=26)
    u, c = np.unique(labels_out, return_counts=True)
    
    label_info[label]['cc3d'] = [u,c]
    
    
    if len(u) > 2:
        continue
        
    mask = labels_out == 1
    
    
    X,Y,Z = label_tensor.shape
    
    xyz_grid = np.stack(np.meshgrid(np.arange(X), np.arange(Y), np.arange(Z), indexing='ij'), -1)
    xyz_grid = xyz_grid[mask]
    center = xyz_grid.mean(0).astype(int)
    
    dx = (np.sum(mask, axis=(1,2)) > 0).sum()
    dy = (np.sum(mask, axis=(0,2)) > 0).sum()
    dz = (np.sum(mask, axis=(0,1)) > 0).sum()
    
    label_info[label]['d_s'] = [dx,dy,dz]
    
    label_info[label]['center'] = center
    
    

  0%|                                                                                                                                         | 0/92 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [23]:
label_info['1']

{'cc3d': [array([0, 1], dtype=uint16), array([22899197,    17011])],
 'd_s': [35, 60, 35],
 'center': array([163, 206,  70])}

In [4]:
# np.save('labels_info',label_info)

In [10]:
# prod_d = [np.prod(d) for d in d_s.values()]
# max_d = [max(d) for d in d_s.values()]

 - Take a look at the FCD volume size distribution
 - Take all crops fully covering FCD
 - Classify from other crops, compare statistics and all